In [ ]:
# user= root 
# password = 7985


In [3]:
import mysql.connector as connection 


In [4]:
try:
    mydb=connection.connect(host="localhost",user="root",password="7985",use_pure=True)
    print(mydb.is_connected())                                   # To check if connection is established with the Database
    #mydb.close()
except Exception as e:
    print(str(e))

True


In [5]:
# Getting the default Databases present in MySQL workbench 

query="SHOW DATABASES"         # Writting the query 
cursor=mydb.cursor()           # creating a cursor to execute the queries 
cursor.execute(query)          # Executing the query
print(cursor.fetchall())       # Fetching the query result

[("b'mavenfuzzyfactory'",), ('demo',), ('india',), ('information_schema',), ('mavenfuzzyfactory',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]


In [6]:
# creating a new database 

query= "create database MyDatabase"
cursor=mydb.cursor()           
cursor.execute(query)          
print("Database created !")

Database created !


In [4]:
# # Deleting a Database
# query= "drop database MyDatabase"
# cursor=mydb.cursor()           
# cursor.execute(query)          
# print("Database Deleted !")

Database Deleted !


In [5]:
mydb.close()           #To shutdown the database from python side 

In [5]:
# creating table to the database "MyDatabase"
# Dont miss putting the database name in the connect function argument
try:
    mydb=connection.connect(host="localhost",database="MyDatabase",user="root",password="7985",use_pure=True)
    print(mydb.is_connected())                                   # To check if connection is established with the Database
    #mydb.close()
except Exception as e:
    print(str(e))

1049 (42000): Unknown database 'mydatabase'


In [9]:
# Table creation
query="create table Students (Student_ID int(10) auto_increment primary key, First_Name Varchar(20) ,Last_Name Varchar(20), Score int(3))"
cursor=mydb.cursor()           
cursor.execute(query)          
print("Table created !")

Table created !


In [10]:
# Inserting Data into the Table

query1="Insert into Students Values(201615001,'Abheek','Nandy',91)"


queries=[query1,query2,query3]
for i in queries:
    cursor=mydb.cursor()           
    cursor.execute(i)
    mydb.commit()                               # For every data insertion we need to commit
print("Data Inserted into the table")

Data Inserted into the table


In [6]:
# Retreiving data from the Database

query="Select * from Students"
cursor=mydb.cursor()           
cursor.execute(query)
print(cursor.fetchall())

[(201615001, 'Abheek', 'Nandy', 91), (201615002, 'Abhranil', 'Roy', 82), (201615003, 'Amalendu', 'Mahali', 61)]


#### Inserting a Whole Dataframe into MYSQL database

In [33]:
# Step1 : Reading the csv file and creating a Dataframe  
import pandas as pd 
df=pd.read_csv("DatabaseDF.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df.head(3)

,District,State,Literacy
0,Thane,Maharashtra,84.53
1,North Twenty Four Parganas,West Bengal,84.06
2,Bangalore,Karnataka,87.67


In [34]:
def Strip(df):
    for i in range(df.shape[0]):
        df.State[i]=df.State[i][1:]
        df.District[i]=df.District[i][1:]
        
    
Strip(df)      

C:\Users\Krishnendu\AppData\Local\Temp\ipykernel_6296\3172979591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.State[i]=df.State[i][1:]
C:\Users\Krishnendu\AppData\Local\Temp\ipykernel_6296\3172979591.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.District[i]=df.District[i][1:]


In [15]:
# # Step2 : Creating Database to store the data

mydb=connection.connect(host="localhost",user="root",password="7985",use_pure=True)
query= "create database India"
cursor=mydb.cursor()           
cursor.execute(query)          
print("Database 'India' created !")

Database 'India' created !


In [36]:
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
cursor=db.cursor()

In [37]:
# Step3 : Create a Table to store the data 

query="create table Literacy (ID int(10) auto_increment primary key, District Varchar(50) ,State Varchar(30), Literacy float(5))"
cursor.execute(query)          
print("Table 'Literacy' created !")

Table 'Literacy' created !


In [ ]:
# # To delete a Table
# query="drop table Literacy"
# cursor=db.cursor()           
# cursor.execute(query)          
# print("table Deleted !")

In [38]:
# Step 4 : Inserting data into the Table

db=connection.connect(host="localhost",database='india',user="root",password="7985",use_pure=True)
cursor=db.cursor()

# cols = "`,`".join(['ID']+[str(i) for i in df.columns.tolist()])
colvalues=[]
for i,row in df.iterrows():
    colvalues.append(tuple([i+1]+list(row)))

# Insert DataFrame recrds one by one.
for i in colvalues:
    query = "INSERT INTO Literacy VALUES{}".format(i)
    cursor.execute(query)
    db.commit()

In [39]:
# Step5 : Retreiving the same data from the MYSQLdatabase 
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
query="Select * from literacy"
df=pd.read_sql(query,db)
df.head(3)

,ID,District,State,Literacy
0,1,Thane,Maharashtra,84.53
1,2,North Twenty Four Parganas,West Bengal,84.06
2,3,Bangalore,Karnataka,87.67


In [40]:
# Other Queries : Groupby , to get the number of districts for each state in Descending order
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
query="select State,count(District) as Districts from literacy group by State order by count(District) desc"
df=pd.read_sql(query,db)
df.head(3)

,State,Districts
0,Uttar Pradesh,71
1,Madhya Pradesh,50
2,Bihar,38


In [41]:
# Sorting by average Literacy (Descending)
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
query="select State,Avg(Literacy) from literacy group by State order by Avg(Literacy) desc"
df=pd.read_sql(query,db)
df.head(3)

,State,Avg(Literacy)
0,Kerala,93.694999
1,Lakshadweep,91.849998
2,Mizoram,89.361251


In [42]:
# Sorting by average Literacy (Descending)
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
query="select State,Avg(Literacy) from literacy group by State order by Avg(Literacy) desc limit 5"
df=pd.read_sql(query,db)
df

,State,Avg(Literacy)
0,Kerala,93.694999
1,Lakshadweep,91.849998
2,Mizoram,89.361251
3,Goa,88.579998
4,Puducherry,87.457502


In [46]:
# Sorting by average Literacy (Descending)
db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
cursor=db.cursor()
query="SELECT District from literacy where State='West Bengal'"
cursor.execute(query)
print(cursor.fetchall())


[('North Twenty Four Parganas',), ('South Twenty Four Parganas',), ('Barddhaman',), ('Murshidabad',), ('Paschim Medinipur',), ('Hugli',), ('Nadia',), ('Purba Medinipur',), ('Haora',), ('Kolkata',), ('Maldah',), ('Jalpaiguri',), ('Bankura',), ('Birbhum',), ('Uttar Dinajpur',), ('Puruliya',), ('Koch Bihar',), ('Darjiling',), ('Dakshin Dinajpur',)]


In [37]:
# Printing the District with most literacy in West Bengal 

db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
cursor=db.cursor()
query="SELECT District from literacy where State=' West Bengal' order by Literacy limit 1"
cursor.execute(query)
print(cursor.fetchall())

[(' Uttar Dinajpur',)]


In [47]:
# Print the District and state with Least Literacy Rate 

db=connection.connect(host="localhost",database='India',user="root",password="7985",use_pure=True)
cursor=db.cursor()
query="Select District,State from literacy order by Literacy limit 1"
cursor.execute(query)
print(cursor.fetchall())

[(' Alirajpur', ' Madhya Pradesh')]
